In [198]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import random
import pandas as pd

In [200]:
id_ = input('아이디를 입력하세요:   ')
password_ = input('비밀번호를 입력하세요:   ')
search = input('검색할 내용을 입력 하세요:   ')
search_count = int(input('몇건을 검색 할까요:   '))

아이디를 입력하세요:   
비밀번호를 입력하세요:   
검색할 내용을 입력 하세요:   
몇건을 검색 할까요:   1


In [204]:
chrome_path = 'C:\chromedriver\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
url = 'https://www.instagram.com/'
driver.get(url)
time.sleep(3)

## 한꺼번에 입력하거나 자주 실행하면 막힘.....ㅎㅎ
element = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').click()
for i in id_:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))

element = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').click()
for i in password_:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))
time.sleep(3)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(2)

## 자동로그인 설정
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

## 알림설정
driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
time.sleep(2)

driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[1]/div').click()
element = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')

for i in search:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))

time.sleep(2)
element.send_keys('\n')
time.sleep(3)
element.send_keys('\n')

time.sleep(6)
title_=[]
hash_tag = []
txt_ = []
img=[]

driver.find_element_by_class_name('_9AhH0').click()
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

try:
    title_text = soup.find('a',class_='O4GlU').get_text()
    title_.append(title_text)
except:
    pass

txt_text = soup.select_one('div.C4VMK>span').get_text()
txt_.append(txt_text)

review_soup= soup.select('a.xil3i')
a = ''
for i in review_soup:
    a += i.get_text()+' '
hash_tag.append(a)


In [ ]:
print(hash_tag)
print(title_)
print(txt_)

In [152]:
a = soup.find('a',class_='O4GlU').get_text()

In [155]:
review_soup= soup.select('a.xil3i')

In [156]:
review_soup

[<a class="xil3i" href="/explore/tags/nikon/" tabindex="0">#Nikon</a>,
 <a class="xil3i" href="/explore/tags/nikonz6/" tabindex="0">#nikonz6</a>,
 <a class="xil3i" href="/explore/tags/allaboutnikon/" tabindex="0">#ALLaboutNIKON</a>,
 <a class="xil3i" href="/explore/tags/web04/" tabindex="0">#Web04</a>,
 <a class="xil3i" href="/explore/tags/changwoncity/" tabindex="0">#changwonCITY</a>,
 <a class="xil3i" href="/explore/tags/landscape/" tabindex="0">#Landscape</a>,
 <a class="xil3i" href="/explore/tags/웅천읍성/" tabindex="0">#웅천읍성</a>,
 <a class="xil3i" href="/explore/tags/웅성/" tabindex="0">#웅성</a>,
 <a class="xil3i" href="/explore/tags/읍성/" tabindex="0">#읍성</a>,
 <a class="xil3i" href="/explore/tags/해자/" tabindex="0">#해자</a>,
 <a class="xil3i" href="/explore/tags/치와문루/" tabindex="0">#치와문루</a>,
 <a class="xil3i" href="/explore/tags/창원시티/" tabindex="0">#창원시티</a>,
 <a class="xil3i" href="/explore/tags/진해구/" tabindex="0">#진해구</a>,
 <a class="xil3i" href="/explore/tags/창원여행/" tabindex="0">#창원여행

In [32]:
driver.find_element_by_class_name('_9AhH0').click()

C:\Users\LIMDON~1\AppData\Local\Temp/ipykernel_41172/1583476414.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('_9AhH0').click()


In [183]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

review_soup= soup.select_one('div.C4VMK>span').get_text()
review_soup

'진해 웅천 읍성에서 만난 옹성옹성이란성문앞에 설치되는 시설물(성벽)인데 그 형상이 항아리 모양을 하고 있어 붙여진 이름. 성문을 부수고 침입하려는 적군을 측면과 후방에서 공격할 수 있다. 포위된 상태를 만들어 전방위로 공격할 수 있어 아군에게 유리한 요새 시설이라고 개인적으로 추측옹성을 설명할려면 성곽위에서 내려다보며 촬영해야 이해하기가 쉽다 하지만 구성을 위한 촬영이다보니 들어가는 입구에서 촬영함#Nikon #nikonz6 #ALLaboutNIKON #Web04 #changwonCITY #Landscape #웅천읍성 #웅성 #읍성 #해자 #치와문루 #창원시티 #진해구 #창원여행'

In [192]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

title_text = soup.find('a',class_='O4GlU').get_text()
title_.append(title_text)

In [193]:
title_

['부산핫플레이스']